# Import librairies

In [4]:
import pandas as pd
import holidays
import requests

# Récupérer les vacances scolaires

In [ ]:
# URL de l'API des vacances scolaires
url_vacances = "https://data.education.gouv.fr/api/records/1.0/search/?dataset=fr-en-calendrier-scolaire&q=&rows=1000&sort=start_date&facet=zone&facet=description&facet=start_date&facet=end_date"

# Charger les données
response = requests.get(url_vacances)
if response.status_code == 200 :
    data = response.json()
    records = data['records']

# Convertir en DataFrame
df = pd.json_normalize(records)

df.columns = df.columns.str.replace('^fields.', '', regex=True)

# Filtrer la Zone B uniquement
df_zone_b_nantes = df[(df['zones'] == 'Zone B') & (df['location'] == 'Nantes')].copy()

# Filtrer années 2025 et 2026
df_year_filtered = df_zone_b_nantes[(df_zone_b_nantes['annee_scolaire'] == '2024-2025') | (df_zone_b_nantes['annee_scolaire'] == '2025-2026')].copy()

# Garder l'essentiel
df_vacances = df_year_filtered[['start_date', 'end_date', 'location', 'annee_scolaire', 'description']].copy()
df_vacances['end_date'] = pd.to_datetime(df_vacances['end_date']).dt.date
df_vacances['start_date'] = pd.to_datetime(df_vacances['start_date']).dt.date


In [40]:
df_vacances

,start_date,end_date,location,annee_scolaire,description
28,2026-07-03,2026-07-03,Nantes,2025-2026,Début des Vacances d'Été
58,2026-05-13,2026-05-17,Nantes,2025-2026,Pont de l'Ascension
79,2026-04-10,2026-04-26,Nantes,2025-2026,Vacances de Printemps
111,2026-02-13,2026-03-01,Nantes,2025-2026,Vacances d'Hiver
150,2025-12-19,2026-01-04,Nantes,2025-2026,Vacances de Noël
162,2025-10-17,2025-11-02,Nantes,2025-2026,Vacances de la Toussaint
197,2025-07-04,2025-08-31,Nantes,2024-2025,Vacances d'Été
213,2025-07-04,2025-08-28,Nantes,2024-2025,Vacances d'Été
274,2025-05-28,2025-06-01,Nantes,2024-2025,Pont de l'Ascension
329,2025-04-04,2025-04-21,Nantes,2024-2025,Vacances de Printemps


In [52]:
# On génère une colonne avec les plages de dates
df_vacances['dates'] = df_vacances.apply(
    lambda row: pd.date_range(start=row['start_date'], end=row['end_date']).tolist(),
    axis=1
)

df_vacances_day_by_day = df_vacances.explode('dates')

df_vacances_final = df_vacances_day_by_day[['dates', 'description']].reset_index(drop=True)


In [53]:
df_vacances_final

,dates,description
0,2026-07-03,Début des Vacances d'Été
1,2026-05-13,Pont de l'Ascension
2,2026-05-14,Pont de l'Ascension
3,2026-05-15,Pont de l'Ascension
4,2026-05-16,Pont de l'Ascension
...,...,...
258,2024-10-30,Vacances de la Toussaint
259,2024-10-31,Vacances de la Toussaint
260,2024-11-01,Vacances de la Toussaint
261,2024-11-02,Vacances de la Toussaint


# Récupérer les jours feriés

In [54]:
# Récupération des jours fériés
fr_holidays = holidays.France(years=[2024,2025,2026])
jours_feries_data = [{'dates': date, 'description': name} for date, name in sorted(fr_holidays.items())]
df_feries = pd.DataFrame(jours_feries_data)

In [55]:
df_feries

,dates,description
0,2024-01-01,New Year's Day
1,2024-04-01,Easter Monday
2,2024-05-01,Labor Day
3,2024-05-08,Victory Day
4,2024-05-09,Ascension Day
5,2024-05-20,Whit Monday
6,2024-07-14,National Day
7,2024-08-15,Assumption Day
8,2024-11-01,All Saints' Day
9,2024-11-11,Armistice Day


## Concat

In [58]:
df_final = pd.concat([df_feries,df_vacances_final], axis =0)

In [59]:
df_final.head()

,dates,description
0,2024-01-01,New Year's Day
1,2024-04-01,Easter Monday
2,2024-05-01,Labor Day
3,2024-05-08,Victory Day
4,2024-05-09,Ascension Day
